In [38]:
import cv2
import numpy as np
import os
import glob
import math

In [39]:
def cut_detected_lines(image, max_lines_to_break=5):

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)


    lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=100, minLineLength=30, maxLineGap=10)

    final_image = image.copy()
    mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)

    if lines is None:
        return final_image, mask

    np.random.shuffle(lines)

    count = 0
    for line in lines:
        if count >= max_lines_to_break:
            break

        x1, y1, x2, y2 = line[0]

        length = math.hypot(x2 - x1, y2 - y1)
        if length < 20: continue

        dx = x2 - x1
        dy = y2 - y1
        angle = math.atan2(dy, dx)


        try:
            sample_x = int(x1 + 5)
            sample_y = int(y1 + 5)
            sample_x = min(max(sample_x, 0), image.shape[1]-1)
            sample_y = min(max(sample_y, 0), image.shape[0]-1)
            gap_color = image[sample_y, sample_x].tolist()
        except:
            gap_color = (127, 127, 127)

        num_breaks = np.random.randint(1, 4)
        for _ in range(num_breaks):
            t = np.random.uniform(0.2, 0.8)

            cx = int(x1 + t * dx)
            cy = int(y1 + t * dy)

            gap_size = np.random.randint(5, 15)

            gap_start_x = int(cx - (gap_size/2) * math.cos(angle))
            gap_start_y = int(cy - (gap_size/2) * math.sin(angle))
            gap_end_x = int(cx + (gap_size/2) * math.cos(angle))
            gap_end_y = int(cy + (gap_size/2) * math.sin(angle))

            cv2.line(final_image, (gap_start_x, gap_start_y), (gap_end_x, gap_end_y), gap_color, 3)

            cv2.line(mask, (gap_start_x, gap_start_y), (gap_end_x, gap_end_y), 255, 3)

        count += 1

    return final_image, mask

In [40]:
def process_folder(input_folder_path, output_base_path):

    images_out = os.path.join(output_base_path, "broken_line_images")
    masks_out = os.path.join(output_base_path, "broken_line_masks")

    os.makedirs(images_out, exist_ok=True)
    os.makedirs(masks_out, exist_ok=True)

    extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp']
    image_files = []
    for ext in extensions:
        image_files.extend(glob.glob(os.path.join(input_folder_path, ext)))

    print(f"Found {len(image_files)} images. Starting Broken Line processing...")

    for file_path in image_files:
        img = cv2.imread(file_path)
        if img is None: continue
        filename = os.path.basename(file_path)

        try:
            defective_img, mask = cut_detected_lines(img)
            cv2.imwrite(os.path.join(images_out, filename), defective_img)
            cv2.imwrite(os.path.join(masks_out, filename), mask)

        except Exception as e:
            print(f"Error on {filename}: {e}")

    print(f"Finished. Results saved in: {output_base_path}")

In [41]:
print("---------- start of Broken line ----------")

SOURCE_DIR ="D:\dataset\Samsung\Original photos"
OUTPUT_BASE_DIR ="D:\dataset\Samsung\Broken line"
process_folder(SOURCE_DIR, OUTPUT_BASE_DIR)

print("---------- end of Broken line ----------")


---------- start of Broken line ----------
Found 2745 images. Starting Broken Line processing...
Finished. Results saved in: D:\dataset\Samsung\Broken line
---------- end of Broken line ----------
